In [1]:
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize
from PIL import Image
from os import path
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import urlopen
import re
import time
import aiohttp
import asyncio
from pathlib import Path
import asyncio
import sys

sys.path.append(path.abspath('MSc-CS-Project---ColourPaletteExtractor-master/'))
from colourpaletteextractor.model import model
# sys.path.append(path.abspath('old_color_palette_gen/MSc-CS-Project---ColourPaletteExtractor-master/'))
# from colourpaletteextractor.model import model

<!-- ### Tate Collection image scraper -->

In [2]:
# Local file dataset
# df = pd.read_json(r'the-tate-collection.json')
# df = pd.json_normalize(df['fields'])
# df
# recoloured_image, colour_palette, relative_frequencies = model.generate_colour_palette_from_image("image/A00544.jpg")
# colors = model.generate_colour_palette_from_image("image/A00544.jpg")
# colors

In [3]:
# recoloured_image
# colour_palette[0]
# relative_frequencies

In [4]:
# view image
# im = Image.open(requests.get("https://www.tate.org.uk/art/images/work/T/T08/T08333_10.jpg", stream = True).raw)
# im

In [5]:
# def url_get_int(url): 
#     url = url.split('_')[1]
#     url = url.split('.')[0]
#     return int(url)

# def binary_find_largest(arr):
#     temp_list = arr
#     length = len(temp_list)
#     if len(arr) == 1:
#         return temp_list[0]
#     elif url_get_int(temp_list[length - 1]) >= url_get_int(temp_list[length - 2]):
#         del temp_list[length - 2]
#     elif url_get_int(temp_list[length - 1]) <= url_get_int(temp_list[length - 2]):
#         del temp_list[length - 1]
    
#     return binary_find_largest(temp_list)
    
# def get_best_imageURL(number):
#     url = df['url'][number]
#     accession_number = df['accession_number'][number]
    
#     page = requests.get(url)
#     soup = BeautifulSoup(page.content, 'html.parser')
#     attrs = []
#     page.close()
    
#     for node in soup.findAll('img'):
#         attrs += list(node.attrs.values())
        
#     temp_images = [k for k in attrs if accession_number in k]
    
#     # returns NaN if no image avilable
#     if len(temp_images) == 0:
#         return np.nan
#     else:
#         return binary_find_largest(temp_images)

In [6]:
# new_df = pd.DataFrame(columns = ['img_URL', 'Colour_Palettes'])

In [7]:
# def set_df_img_url_range(start, last):
#     timer = time.process_time()
    
#     for counter in range(start, last + 1):
#         try:
#             url = get_best_imageURL(counter)
#             if url is not np.nan:
#                 savepath = Path("image/"+ str(df["accession_number"][counter]) + ".jpg")
#                 if not savepath.is_file():
#                     urllib.request.urlretrieve(url, savepath)
                
#                 if len(new_df) == counter:
#                     colors = model.generate_colour_palette_from_image(savepath)
#                 elif new_df.loc[counter]['Colour_Palettes'] is np.nan:
#                     colors = model.generate_colour_palette_from_image(savepath)

#                 new_df.loc[counter] = [url, colors]

#             else:
#                 new_df.loc[counter] = [url, np.nan]   

#         except:
#             print(str(counter))
#             counter -= 1
#             continue
    
#     print(time.process_time() - timer)

In [8]:
# set_df_img_url_range(0, 100)

In [9]:
# new_df

# ArtUK web scraping

In [10]:
# Link that has search settings set to the following
# Location: Scotland, Type: Painting, Topic: Seas and Coasts, Has image ticked, old to new
url = "https://artuk.org/discover/artworks/view_as/grid/search/has_image:on--region:scotland--work_type:painting--category:seas-and-coasts/sort_by/date_earliest/order/asc/page/1"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
page.close()
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>
												Art UK | Discover Artworks 									</title>
<meta content="" name="author"/>
<meta content="Discover artworks, explore venues and meet artists. Art UK is the online home for every public collection in the UK. Featuring over 275,000 artworks by over 50,000 artists." name="description"/>
<meta content="art, uk, art uk, nsw-rlrbwj98djuzr8je9lkikzom-numnw-a0xd1tjxmjqpgxb-2-8x1-ujelx-g77bi-fw64rhobgu0-7v3t-glh32vmekcliuy70l4olzmeqmwsbzhe9mc2msn8yphihiy" name="keywords"/>
<meta content="width=320, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="https://artuk.org/skins/artuk/img/logo-share-red-bg.jpg" property="og:image"/>
<meta content="https://artuk.org/skins/artuk/img/logo-share-red-bg.jpg" property="twitter:image"/>
<meta content="image/jpg" property="og:image:type"/>
<meta content="1200" property="og:image:wi

In [11]:
# get the number of images on the page currently
def num_ofimages_on_pg(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    page.close()
    nodes = [[node] for node in soup.findAll('li', {"class": "item artwork icons"})]
    return int(len(nodes))

# get the url that shows all image from the search
def get_URL_All_img(url):
    count = soup.find('span', {"class": "count"})
    num = count.find('span', {"class": "num"})
    max_page = int(int(num.contents[0]) / 20)
    return url.replace(url.split("/")[-1], str(max_page + 1))

In [12]:
# num_ofimages_on_pg(url)

In [13]:
# temp_url = get_URL_All_img(url)
# print(temp_url)
# num_ofimages_on_pg(temp_url)

In [14]:
# Artuk page with all images loaded
all_img_url = get_URL_All_img(url)
page = requests.get(all_img_url)
all_results = BeautifulSoup(page.content, 'html.parser')
page.close()
all_results

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>
												Art UK | Discover Artworks 									</title>
<meta content="" name="author"/>
<meta content="Discover artworks, explore venues and meet artists. Art UK is the online home for every public collection in the UK. Featuring over 275,000 artworks by over 50,000 artists." name="description"/>
<meta content="art, uk, art uk, nsw-rlrbwj98djuzr8je9lkikzom-numnw-a0xd1tjxmjqpgxb-2-8x1-ujelx-g77bi-fw64rhobgu0-7v3t-glh32vmekcliuy70l4olzmeqmwsbzhe9mc2msn8yphihiy" name="keywords"/>
<meta content="width=320, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="https://artuk.org/skins/artuk/img/logo-share-red-bg.jpg" property="og:image"/>
<meta content="https://artuk.org/skins/artuk/img/logo-share-red-bg.jpg" property="twitter:image"/>
<meta content="image/jpg" property="og:image:type"/>
<meta content="1200" property="og:image:wi

In [15]:
# attributes of all images on the page
all_images = [node for node in all_results.findAll('li', {"class": "item artwork icons"})]
len(all_images)

455

### Artuk dataframe

#### create dataset (first time)

In [16]:
# removes all '\t' & '\n'
def remove_weird_chars(string):
    clean_string = string.replace('\n', '')
    clean_string = clean_string.replace('\t', '')
    return clean_string.strip()

def get_title(n):
    title = all_images[n].find('span', {"class": "title"})
    cleaned_title = remove_weird_chars(title.contents[0])
    if cleaned_title == 'Untitled':
        return np.nan
    return cleaned_title

def get_URL(n):
    embeded_url = all_images[n].find('a', {"class": ""})
    embeded_url = embeded_url.attrs['href']
    return embeded_url

def split_name_year(string):
    if '(' not in string: 
        return [string.strip()]
    split_string = string.split('(')
    split_string[0] = split_string[0].strip()
    split_string[1] = ('(' + split_string[1]).strip()
    return split_string

def get_artist_name(n):
    tag = all_images[n].find('span', {"class": "artist"})
    tag = remove_weird_chars(tag.contents[0])
    
    
    if tag == "unknown artist":
        return np.nan

    artist_name_and_date = split_name_year(tag)
    artist_name = artist_name_and_date[0]
    return artist_name

def get_art_year(n):
    tag = all_images[n].find('span', {"class": "date"})
    if tag is None:
        return np.nan
    tag = remove_weird_chars(tag.contents[0])
    return str(tag)

def dimension_text_to_num(string):
    clean_string = string.replace('cm', '')
    clean_string = clean_string.replace('H', '')
    clean_string = clean_string.replace('W', '')
    clean_string = clean_string.replace('(E)', '')
    clean_string = clean_string.split('x')
    return [float(clean_string[0].strip()), float(clean_string[1].strip())]

def year_to_estimate(creation_year):
    if creation_year is np.nan:
        return np.nan
    
    elif any(x in creation_year for x in ["-", "–"]):
        split_string = creation_year.split('–')
        
        start_filter = filter(str.isdigit, split_string[0])
        start = "".join(start_filter)
        
        end_filter = filter(str.isdigit, split_string[1])
        end = "".join(end_filter)
        
        start = start.strip()
        end = end.strip()
        avg = int(start) + int(end)
        avg = round(avg / 2)
        return str(avg)
    
    elif any(x in creation_year for x in ["th", "C"]):
        numeric_filter = filter(str.isdigit, creation_year)
        numeric_string = "".join(numeric_filter) + "00"
        return numeric_string.strip()
    
    else:
        numeric_filter = filter(str.isdigit, creation_year)
        numeric_string = "".join(numeric_filter)
        return numeric_string.strip()

# get_all_other & set_all_other sets: the enlarged imaged url, height and width, Accession number, medium
# as they are all in the embedded url, getting them all at once to increase speed
async def get_all_other(session, n):
    async with session.get(art_uk_df['url'][n]) as resp:
        page = await resp.text()
        embeded_page =  BeautifulSoup(page, 'html.parser')
        
        all_info = embeded_page.find('div', {'class' : 'masonry_details'})
        items = all_info.findAll('div', {'class' : 'masonry-item'})
        
        for tag in items:
            # Medium
            if tag.findNext('h5').text == "Medium":
                medium = tag.findNext('h5').findNext('p').text
                
            # Height and Width
            elif tag.findNext('h5').text == "Measurements":
                height = dimension_text_to_num(tag.findNext('h5').findNext('p').text)[0]
                width = dimension_text_to_num(tag.findNext('h5').findNext('p').text)[1]

            # Accession Number
            elif tag.findNext('h5').text == "Accession number":
                accession_number = tag.findNext('h5').findNext('p').text.replace('/', '_').replace(':', '_')
        
        # image_url
        if embeded_page.find('img', {"alt": art_uk_df['Title'][n]}) is None:
            enlarged_img = embeded_page.find('img', {"alt": "Untitled"})
            enlarged_img = enlarged_img.attrs['src'] 
        else:
            enlarged_img = embeded_page.find('img', {"alt": art_uk_df['Title'][n]})
            enlarged_img = enlarged_img.attrs['src'] 
        return [medium, height, width, accession_number, enlarged_img]
    
async def set_all_other():
     async with aiohttp.ClientSession() as session:
        tasks = []
        
        for number in range(len(all_images)):
            tasks.append(asyncio.ensure_future(get_all_other(session, number)))
                
        tasks = await asyncio.gather(*tasks)
        
        # image_url
        art_uk_df['Medium'] = [tasks[n][0] for n in range(len(all_images))]
        art_uk_df['height_cm'] = [tasks[n][1] for n in range(len(all_images))]
        art_uk_df['width_cm'] = [tasks[n][2] for n in range(len(all_images))]
        art_uk_df['Accession_number'] = [tasks[n][3] for n in range(len(all_images))]
        art_uk_df['img_URL'] = [tasks[n][4] for n in range(len(all_images))]
        
def create_artuk_df(): 
    # Title
    art_uk_df['Title'] = [get_title(n) for n in range(len(all_images))]
    
    # embedded url
    art_uk_df['url'] = [get_URL(n) for n in range(len(all_images))]
    
    # Artist Name
    art_uk_df['Artist'] = [get_artist_name(n) for n in range(len(all_images))]
    
    # Art Creation Date
    art_uk_df['Creation_Date'] = [get_art_year(n) for n in range(len(all_images))]
    
    art_uk_df['estimate_year'] = [year_to_estimate(art_uk_df['Creation_Date'][n]) for n in range(len(all_images))]

##### Get color palettes

In [17]:
async def get_colors(n):
    savepath = Path("ArtUK_scotland_seas_coasts/" + str(art_uk_df["Accession_number"][n]) + ".jpg")
    if savepath.is_file():
        recoloured_image, colour_palette, relative_frequencies = model.generate_colour_palette_from_image(savepath)
        await asyncio.sleep(1)
        return recoloured_image, colour_palette, relative_frequencies
    else:
        return np.nan, np.nan, np.nan
    
async def set_all_colors():
    tasks = []
    for number in range(len(all_images)):
        tasks.append(asyncio.ensure_future(get_colors(number)))

    await asyncio.gather(*tasks)
    art_uk_df['recoloured_image'] = [tasks[n].result()[0] for n in range(len(all_images))]
    art_uk_df['colour_palette'] = [tasks[n].result()[1] for n in range(len(all_images))]
    art_uk_df['relative_frequencies'] = [tasks[n].result()[2] for n in range(len(all_images))]
# timer = time.process_time()
# await set_all_colors()
# print(time.process_time() - timer)
# art_uk_df.to_csv(artUK_collection, index = False)

##### import or create new df

In [18]:
artUK_collection = Path(r'artUK-collection.csv')

# if csv exist, then just import
if artUK_collection.is_file():
    art_uk_df = pd.read_csv(artUK_collection)
    
# create dataset
else:
    # new df, set new columns
    art_uk_df = pd.DataFrame(columns = ['Title', 'Artist', 'Creation_Date', 'estimate_year', 'url', 
                                        'img_URL', 'Medium', 'height_cm', 'width_cm', 'Accession_number', 
                                        'recoloured_image', 'colour_palette', 'relative_frequencies'])
    
    timer = time.process_time()
    # set meta data
    create_artuk_df()
    await set_all_other()
    
    # set color palettes
    await set_all_colors()
    
    # inital set up time taken:
    print(time.process_time() - timer)
    
    # export
    art_uk_df.to_csv(artUK_collection, index = False)

Added image!
Generating colour palette for image: Tab_0
Retrieving image...
Generating colour palette...
60 CIELAB cubes generated...
Assigning each pixel to the appropriate CIELAB cube...
--- 0.5365691184997559 seconds for Python cube assignment loop ---
Secondary pixel count threshold: 1294.6499999999999
3509 0 10357.199999999999 3509
2269 0 10357.199999999999 2269
4252 0 10357.199999999999 6702
1403 0 10357.199999999999 1403
2427 0 10357.199999999999 5819
2412 0 10357.199999999999 2412
1373 0 10357.199999999999 2649
1571 0 10357.199999999999 1571
Number of relevant colours: 15
Updating pixel colours...
--- 0.8710007667541504 seconds for Python pixel colour update loop ---
New colour
[7.79234411 3.92349431 7.77153173]
[32 21 10]
New colour
[17.66811508  6.99298452 22.05663905]
[61 39  9]
New colour
[ 7.62510473 -1.10740401  8.29697428]
[26 23  8]
New colour
[17.55653237 -2.81897001 21.64503595]
[48 43  9]
New colour
[28.10228811  3.42516272 15.07418154]
[79 64 43]
New colour
[31.5912

--- 0.9835224151611328 seconds for Python pixel colour update loop ---
New colour
[14.85397438  4.01897964 10.73063151]
[48 35 22]
New colour
[18.06349639  7.29479724 22.26117516]
[62 40 10]
New colour
[14.25637726 -1.16616965 11.50317114]
[40 36 20]
New colour
[25.02866179  6.10399809 15.79401335]
[76 55 36]
New colour
[31.31168605  8.96641976 26.07189308]
[98 68 33]
New colour
[47.13255292  6.83139487 28.76806724]
[138 106  63]
New colour
[48.94201324 12.48101898 42.84127186]
[154 107  41]
New colour
[51.59887573 -1.325488   23.25723139]
[135 122  83]
New colour
[63.40213159  7.83691209 33.99397837]
[186 147  93]
(565, 800, 3) 9

---------------
Colour Palette:
[48 35 22]
[98 68 33]
[138 106  63]
[76 55 36]
[40 36 20]
[186 147  93]
[135 122  83]
[62 40 10]
[154 107  41]
---------------
Added image!
Generating colour palette for image: Tab_0
Retrieving image...
Generating colour palette...
40 CIELAB cubes generated...
Assigning each pixel to the appropriate CIELAB cube...
--- 0.757499

4243 0 11425.8 4243
2034 0 11425.8 2034
2118 0 11425.8 2118
1858 0 11425.8 1858
Number of relevant colours: 13
Updating pixel colours...
--- 0.9110250473022461 seconds for Python pixel colour update loop ---
New colour
[12.29582431  3.52501278 11.17754465]
[42 30 16]
New colour
[17.80754309  7.39912698 22.50326195]
[62 39  8]
New colour
[14.4619392  -1.55627493 11.2427496 ]
[40 37 21]
New colour
[27.61036632  3.60493461 12.01850716]
[77 63 47]
New colour
[27.56838106  8.55333174 26.8224788 ]
[88 59 23]
New colour
[31.50572266 26.53706995 31.10522357]
[121  55  26]
New colour
[30.1710795  -2.55710342  9.36111075]
[73 72 56]
New colour
[26.95680473 -2.39896917 22.30369726]
[71 64 29]
New colour
[45.8311509  11.38445852 32.00745227]
[141 100  55]
New colour
[46.75580247 14.66266348 45.08045444]
[151 100  31]
New colour
[47.24615639 29.56334951 35.33843827]
[170  90  53]
New colour
[48.69933785 33.65137022 43.95081416]
[181  90  41]
New colour
[44.40742932 -2.2650322   9.45057931]
[108 106

125 CIELAB cubes generated...
Assigning each pixel to the appropriate CIELAB cube...
--- 0.6715033054351807 seconds for Python cube assignment loop ---
Secondary pixel count threshold: 1746.0
4058 0 13968.0 4058
8554 0 13968.0 8554
13568 0 13968.0 13568
8447 0 13968.0 8447
2569 0 13968.0 2569
1908 0 13968.0 1908
Number of relevant colours: 16
Updating pixel colours...
--- 1.276000738143921 seconds for Python pixel colour update loop ---
New colour
[11.14288933  2.33175029 10.95219969]
[38 28 14]
New colour
[17.53938809  4.35694065 23.20903106]
[58 40  5]
New colour
[ 9.87491725 -3.61259485  8.8511197 ]
[27 28 14]
New colour
[17.52477318 -5.79421465 22.69608511]
[45 45  6]
New colour
[28.39585477  4.31782833 29.52447199]
[85 64 19]
New colour
[26.16342789 -5.51195357 16.08009662]
[62 64 37]
New colour
[29.86794924 -6.75863735 28.45650446]
[74 72 24]
New colour
[50.83851399  5.85045308 28.35251378]
[146 116  73]
New colour
[48.17046327 -6.36203658 30.67852358]
[121 116  61]
New colour
[4

125 CIELAB cubes generated...
Assigning each pixel to the appropriate CIELAB cube...
--- 1.2014966011047363 seconds for Python cube assignment loop ---
Secondary pixel count threshold: 3568.5
4031 0 28548.0 22209
8438 0 28548.0 8438
16017 0 28548.0 16017
4826 0 28548.0 4826
4279 0 28548.0 16851
15909 0 28548.0 15909
3783 3784 28548.0 3784
Number of relevant colours: 13
Updating pixel colours...
--- 2.208519458770752 seconds for Python pixel colour update loop ---
New colour
[14.91770084  5.62971303 13.91716182]
[51 34 18]
New colour
[17.94693806  7.91412041 22.46959152]
[62 39  9]
New colour
[27.1092806   5.17337771 16.62077222]
[80 61 39]
New colour
[30.08404199  8.57778278 27.1337247 ]
[94 65 28]
New colour
[32.65106756 23.65689038 32.20336176]
[121  60  26]
New colour
[49.595177    6.51309243 29.42953464]
[144 113  68]
New colour
[48.88679038 13.49623836 43.32998087]
[155 106  40]
New colour
[46.84907458 26.19587728 45.792724  ]
[167  92  31]
New colour
[53.62658668 -3.86584104 23.9

...

100 CIELAB cubes generated...
Assigning each pixel to the appropriate CIELAB cube...
--- 1.5399763584136963 seconds for Python cube assignment loop ---
Secondary pixel count threshold: 4347.0
6235 0 34776.0 20718
9341 0 34776.0 9341
19866 0 34776.0 19866
4697 0 34776.0 19707
4467 0 34776.0 4467
18348 0 34776.0 21618
6734 0 34776.0 6734
5722 0 34776.0 5722
6442 0 34776.0 21902
3662 4716 34776.0 4716
Number of relevant colours: 21
Updating pixel colours...
--- 2.09100079536438 seconds for Python pixel colour update loop ---
New colour
[13.34385161  3.03616188  3.82145202]
[41 33 29]
New colour
[15.74024822  2.02483845 -4.66636314]
[39 39 46]
New colour
[14.92208295 -2.65514436  3.11238604]
[36 38 33]
New colour
[16.31111343 -2.28273239 -4.200406  ]
[33 42 46]
New colour
[31.49380396  5.54105492  7.88648319]
[87 71 62]
New colour
[35.11961661 12.93132086 22.5919357 ]
[112  74  47]
New colour
[27.26257075  1.53999212 -4.98809345]
[63 64 72]
New colour
[29.29030199 -2.4035381   3.54813805]


210 CIELAB cubes generated...
Assigning each pixel to the appropriate CIELAB cube...
--- 0.8425021171569824 seconds for Python cube assignment loop ---
Secondary pixel count threshold: 2456.7599999999998
6079 0 19654.079999999998 6079
3602 0 19654.079999999998 3602
6051 0 19654.079999999998 6051
6632 0 19654.079999999998 6632
11070 0 19654.079999999998 11070
8161 0 19654.079999999998 8161
7553 7553 19654.079999999998 7553
0 6145 19654.079999999998 6145
Number of relevant colours: 12
Updating pixel colours...
--- 1.4740324020385742 seconds for Python pixel colour update loop ---
New colour
[ 36.73704884 -22.37268832  29.44152637]
[64 95 36]
New colour
[ 34.9161943  -18.06446715  23.59574056]
[65 89 43]
New colour
[ 51.17417366 -24.54277464  32.85185804]
[ 98 132  63]
New colour
[ 53.7119866  -27.78227278  42.2057292 ]
[102 140  51]
New colour
[ 47.66501955 -18.00982829  24.57017303]
[ 97 120  71]
New colour
[ 68.53649645 -24.3783674   34.05735153]
[144 178 104]
New colour
[ 68.03348787 

60 CIELAB cubes generated...
Assigning each pixel to the appropriate CIELAB cube...
--- 0.6194643974304199 seconds for Python cube assignment loop ---
Secondary pixel count threshold: 1755.0
2947 0 14040.0 5038
5284 0 14040.0 7475
2672 0 14040.0 2672
3077 0 14040.0 3077
4911 0 14040.0 10818
5394 0 14040.0 5394
3269 0 14040.0 3269
4824 4824 14040.0 4824
Number of relevant colours: 14
Updating pixel colours...
--- 1.0094952583312988 seconds for Python pixel colour update loop ---
New colour
[12.52590901  6.8302584   9.98498471]
[46 29 19]
New colour
[29.75713931  5.47436129 12.36764477]
[85 67 51]
New colour
[31.97533215 12.45858302 25.8348433 ]
[104  67  35]
New colour
[29.40466289 24.58781153 28.29857187]
[112  52  26]
New colour
[32.04997984 -4.82782265  7.96208867]
[73 77 63]
New colour
[51.17271996  2.41840696 10.85387163]
[133 120 104]
New colour
[51.1619425   9.16281585 27.02285173]
[151 115  76]
New colour
[50.62871798 -3.3909592   9.19598849]
[121 122 105]
New colour
[72.6680143

125 CIELAB cubes generated...
Assigning each pixel to the appropriate CIELAB cube...
--- 1.3929955959320068 seconds for Python cube assignment loop ---
Secondary pixel count threshold: 4036.5
28741 0 32292.0 28741
3274 18565 32292.0 18565
Number of relevant colours: 10
Updating pixel colours...
--- 3.0340070724487305 seconds for Python pixel colour update loop ---
New colour
[13.32004971  2.05499151 -3.67430455]
[35 33 39]
New colour
[32.98733683  5.08569022  5.05668855]
[89 75 70]
New colour
[27.71792637  2.44083576 -3.36967462]
[67 64 71]
New colour
[50.1607607   7.43261987  9.23376306]
[138 115 104]
New colour
[54.41543339 13.13798697 26.53315746]
[166 121  85]
New colour
[72.24554001  4.53415972  7.88535754]
[192 174 163]
New colour
[66.06317851 11.35974376 26.45889854]
[196 152 114]
New colour
[74.85367879 -1.93327578  5.33269363]
[185 185 174]
New colour
[83.58763427  3.40276149  8.59260567]
[222 206 192]
New colour
[81.64803304 -1.55468022  4.96383932]
[204 203 194]
(897, 1200, 

--- 1.5089986324310303 seconds for Python pixel colour update loop ---
New colour
[13.65806039  5.9900367  12.09644754]
[48 31 18]
New colour
[17.48010703  9.81785473 23.26136717]
[64 37  6]
New colour
[13.90359363 -2.6642257   9.87679514]
[37 36 22]
New colour
[29.07229634  4.74304459 13.24363961]
[83 65 48]
New colour
[31.85552891  6.97219956 29.7078266 ]
[98 70 27]
New colour
[37.12072095  7.48220065 41.32693299]
[114  82  14]
New colour
[31.25106992 25.17212582 31.16363438]
[118  56  25]
New colour
[29.8360292  -2.45268657 10.23944721]
[73 71 54]
New colour
[32.35670195 -2.22436432 26.99298661]
[86 76 32]
New colour
[49.30653122  3.51498049 13.10404471]
[132 114  95]
New colour
[46.42861517  5.21556307 29.82058281]
[134 106  60]
New colour
[43.69062092  6.41825238 41.88211196]
[131  98  30]
New colour
[53.4204462  -2.97123747  8.19693326]
[128 129 114]
New colour
[48.2772134  -1.92804831 28.86704267]
[128 114  65]
New colour
[52.82251858 -8.43882365 -4.75731536]
[105 131 134]
New c

##### Download all images to one folder

In [19]:
async def get_img(n):
    savepath = Path("ArtUK_scotland_seas_coasts/" + str(art_uk_df["Accession_number"][n]) + ".jpg")
    if not savepath.is_file():
        urllib.request.urlretrieve(art_uk_df['img_URL'][n], savepath)
    await asyncio.sleep(1)
    
async def download_All():
        tasks = []
        for number in range(len(all_images)):
            tasks.append(asyncio.ensure_future(get_img(number)))
        await asyncio.gather(*tasks)

timer = time.process_time()
await download_All()
print(time.process_time() - timer)

0.046875


##### Update

In [20]:
create_artuk_df()
timer = time.process_time()
await set_all_other()
print(time.process_time() - timer)

47.46875


In [21]:
# await set_all_colors()
# print(time.process_time() - timer)
# art_uk_df.to_csv(artUK_collection, index = False)

In [22]:
art_uk_df.to_csv(artUK_collection, index = False)

In [23]:
#view
art_uk_df
# art_uk_df[art_uk_df["Creation_Date"].str.contains("later|after|before", na = False)]

,Title,Artist,Creation_Date,estimate_year,url,img_URL,Medium,height_cm,width_cm,Accession_number,recoloured_image,colour_palette,relative_frequencies
0,The Dream of Joseph,Paolo Fiammingo,c.1587–1596,1592,https://artuk.org/discover/artworks/the-dream-...,https://d3d00swyhr67nd.cloudfront.net/w944h944...,oil on canvas,160.0,116.8,1591,"[[[32, 21, 10], [32, 21, 10], [32, 21, 10], [3...","[[32, 21, 10], [79, 64, 43], [96, 69, 34], [26...","[0.4320212026416406, 0.1342225698065114, 0.112..."
1,A Scottish Warship,NaN,1650–1674,1662,https://artuk.org/discover/artworks/a-scottish...,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,oil on canvas,82.4,98.7,ABDAG001234,"[[[138, 120, 106], [138, 120, 106], [173, 171,...","[[193, 177, 166], [224, 208, 196], [45, 26, 19...","[0.2053546626984127, 0.16702132936507935, 0.10..."
2,Ships in a Calm Sea,Willem van de Velde II,c.1660s,1660,https://artuk.org/discover/artworks/ships-in-a...,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,oil on panel,36.8,49.9,NG 1947,"[[[201, 183, 125], [201, 183, 125], [201, 183,...","[[201, 183, 125], [215, 177, 98], [205, 176, 1...","[0.23980818965517242, 0.19919181034482758, 0.1..."
3,Fishing Boats in a Calm,Willem van de Velde II,1658,1658,https://artuk.org/discover/artworks/fishing-bo...,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,oil on canvas,41.9,56.2,EU0744,"[[[185, 180, 155], [185, 180, 155], [185, 180,...","[[185, 180, 155], [210, 205, 179], [189, 175, ...","[0.3297661803217359, 0.15503367003367002, 0.12..."
4,A Mountainous Coastal Landscape with Classical...,Jean Daret,1661,1661,https://artuk.org/discover/artworks/a-mountain...,https://d3d00swyhr67nd.cloudfront.net/w944h944...,oil on canvas,99.7,138.7,212,"[[[76, 55, 36], [76, 55, 36], [76, 55, 36], [7...","[[48, 35, 22], [98, 68, 33], [138, 106, 63], [...","[0.3337676991150442, 0.2481637168141593, 0.178..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,Estuary,Ralph Cowan,NaN,NaN,https://artuk.org/discover/artworks/estuary-15...,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,oil on board,87.3,114.5,GLAEX A113,"[[[192, 174, 163], [192, 174, 163], [192, 174,...","[[192, 174, 163], [185, 185, 174], [138, 115, ...","[0.4507961724266072, 0.11447324414715719, 0.09..."
451,Cargo for Power,Kenneth M. Sibley,NaN,NaN,https://artuk.org/discover/artworks/cargo-for-...,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,oil on board,30.7,59.5,ABDAG008033,"[[[61, 75, 80], [102, 121, 126], [102, 121, 12...","[[102, 121, 126], [61, 75, 80], [164, 169, 158...","[0.41855353375527427, 0.10980617088607594, 0.1..."
452,"Raised Beaches, Colonsay",John Cunningham,NaN,NaN,https://artuk.org/discover/artworks/raised-bea...,https://d3d00swyhr67nd.cloudfront.net/w944h944...,oil on canvas,73.8,83.8,GLAEX A302,"[[[140, 169, 169], [140, 169, 169], [140, 169,...","[[159, 175, 161], [140, 169, 169], [212, 212, ...","[0.2307903849568872, 0.17454766085909906, 0.08..."
453,Trees in a Golden Field and Sea*,NaN,NaN,NaN,https://artuk.org/discover/artworks/trees-in-a...,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,oil on canvas,44.0,58.0,065466,"[[[181, 208, 224], [181, 208, 224], [181, 208,...","[[239, 217, 120], [181, 208, 224], [166, 196, ...","[0.19112026862026862, 0.15524928774928776, 0.1..."


In [27]:
import webcolors
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.css3_hex_to_names.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name

def colour_palette_to_list(array):
    return
art_uk_df['colour_palette'][0][0]
# requested_colour = (0, 0, 0)
# actual_name, closest_name = get_colour_name(requested_colour)
# print("Actual colour name:", actual_name, ", closest colour name:", closest_name)

array([32, 21, 10], dtype=uint8)